# Load Model

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
#from helpers import *
from pathlib import Path
#from fastai.callbacks.hooks import *
from PIL import ImageFilter
import numpy
from torchvision import transforms
import re
#import torchsummary
from PIL import Image
import sys
from collections import namedtuple
from torchsummary1 import summary
import glob
from dataloader import *
import math

In [ ]:
sys.path.append('../3D-ResNets-PyTorch/')

In [ ]:
import model

# Setting hyperparameters and loading model

In [ ]:
options = {
    "model_depth": 50,
    "model": 'resnet',
    "n_classes": 400,
    "n_finetune_classes": 5,
    "resnet_shortcut": 'B',
    "sample_size": (576,704),
    "sample_duration": 16,
    "pretrain_path": '../3D-ResNets-PyTorch/resnet-50-kinetics.pth',
    "no_cuda": False,
    "arch": 'resnet-50',
    "ft_begin_index": 0
}

opts = namedtuple("opts", sorted(options.keys()))

myopts2 = opts(**options)
#myopts2.model_depth

#generate_model(myopts2)

In [ ]:
from model import generate_model

In [ ]:
int(math.ceil(576 / 32))

In [ ]:
model, parameters = generate_model(myopts2)

In [ ]:
model.parameters()

In [ ]:
parameters

In [ ]:
summary(model, (3,8,576,704))

In [ ]:
# # Parallelize model to multiple GPUs

# print("Using", torch.cuda.device_count(), "GPUs!")

# model1 = nn.DataParallel(model, device_ids = [1,0])
# model1.to(f'cuda:{model.device_ids[1]}')

# Loading the training data

In [ ]:
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
class_paths = [d.path for d in os.scandir(root_dir) if d.is_dir]

transform = transforms.Compose([
    transforms.Resize((576, 704)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_names = ['exp_and','exp_fs','exp','exp_fj','bur']
one_hot_classes = [[1,0,1,0,0],[1,0,0,0,1],[1,0,0,0,0],[1,0,0,1,0],[0,1,0,0,0]]

df = pd.read_csv('./train-valid-splits-video/train.csv')


In [ ]:
bs = 2

In [ ]:
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
    for d in os.scandir(class_path):
        if d.is_dir:
            if d.path in df.videos.values:
                paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
                # Add class idx to paths
                paths = [(p, one_hot_classes[c]) for p in paths]
                class_image_paths.extend(paths)
                end_idx.extend([len(paths)])
                
end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
seq_length = 16

In [ ]:
sampler = MySampler(end_idx, seq_length)

In [ ]:
dataset = MyDataset(
    image_paths=class_image_paths,
    seq_length=seq_length,
    transform=transform,
    length=len(sampler))

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=bs,
    sampler=sampler,
    num_workers=0,
    drop_last = True
)

## Show a batch

In [ ]:
# Get a batch of training data
# inputs, classes = next(iter(loader))
# inputs = inputs.squeeze(dim = 0)

# for j in range(bs):
#     # Make a grid from batch
#     out = torchvision.utils.make_grid(inputs[j])


#     for i, f in enumerate(one_hot_classes):
#         if np.array_equal(classes[j][0].numpy(), np.asarray(f)):
#             title = class_names[i]


#     imshow(out, title=title)

# Training

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from torch.autograd import Variable
import time
import os
import sys
from utils import AverageMeter

In [ ]:
# # Detect devices
# use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda:0")   # use CPU or GPU

In [ ]:
#def train_epoch(epoch, data_loader, model, criterion, optimizer, opt, epoch_logger, batch_logger):
epochs = 3
save_model_path = './save-model-3d/'
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(epochs):
    print('train at epoch {}'.format(epoch))
    model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
#     accuracies = AverageMeter()
    end_time = time.time()
    for i, (inputs, targets) in enumerate(loader):
        data_time.update(time.time() - end_time)
                                                    #cuda(non_blocking=True)
        inputs = inputs.to(device).to(device)      #cuda(non_blocking=True)

        targets = Variable(targets.long()).to(device) 
        targets = targets.squeeze(dim=1)
        #targets = targets.type_as(outputs) #comment in the first try

        inputs = inputs.permute(0,2,1,3,4)
        outputs = model(inputs)
        targets = targets.float()
        loss = F.binary_cross_entropy_with_logits(outputs, targets)
        #acc = calculate_accuracy(outputs, targets)

        losses.update(loss.item(), inputs.size(0))
        #accuracies.update(acc, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        print('Epoch: [{0}][{1}/{2}]\t' #'Acc {acc.val:.3f} ({acc.avg:.3f})'
              'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(
                  epoch,
                  i + 1,
                  len(loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses))

    save_file_path = os.path.join(save_model_path, 'save_{}.pth'.format(epoch))
    states = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        }
    torch.save(states, save_file_path)


# Load saved model

In [ ]:
state_dict = torch.load('./save-model-3d/save_2.pth')['state_dict']
model.load_state_dict(state_dict)

In [ ]:
model

# Loading Validation data

In [ ]:
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
class_paths = [d.path for d in os.scandir(root_dir) if d.is_dir]

transform = transforms.Compose([
    transforms.Resize((576, 704)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_names = ['exp_and','exp_fs','exp','exp_fj','bur']
one_hot_classes = [[0,1,1,0,0],[0,1,0,0,1],[0,1,0,0,0],[0,1,0,1,0],[1,0,0,0,0]]

df = pd.read_csv('./train-valid-splits-video/valid.csv')


In [ ]:
bs = 16

In [ ]:
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
    for d in os.scandir(class_path):
        if d.is_dir:
            if d.path in df.videos.values:
                paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
                # Add class idx to paths
                paths = [(p, one_hot_classes[c]) for p in paths]
                class_image_paths.extend(paths)
                end_idx.extend([len(paths)])
                
end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
seq_length = 16

In [ ]:
sampler = MySampler(end_idx, seq_length)

In [ ]:
dataset = MyDataset(
    image_paths=class_image_paths,
    seq_length=seq_length,
    transform=transform,
    length=len(sampler))

In [ ]:
valid_loader = DataLoader(
    dataset,
    batch_size=bs,
    sampler=sampler,
    num_workers=0,
    drop_last = True
)

In [ ]:
len(valid_loader)

# Validation

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda:0") 

In [ ]:
model.eval()

test_loss = 0
all_y = []
all_y_pred = []
y_pred = []

with torch.no_grad():
    for X, y in valid_loader:
        # distribute data to device
        X, y = X.to(device), y.to(device)
        X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        #y = y.type_as(output) # comment that line the first time and uncomment it after that
        y = y.float()
        output = model(X)
        loss = F.binary_cross_entropy_with_logits(output, y)
        test_loss += loss.item()   
        # sum up batch loss
        y_pred = output.sigmoid()
        # collect all y and y_pred in all batches
        all_y.extend(y)
        all_y_pred.extend(y_pred)


In [ ]:
test_loss /= len(valid_loader.dataset)

# compute accuracy
#all_y = torch.stack(all_y, dim=0)
#all_y_pred = torch.stack(all_y_pred, dim=0)
#test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())
accuracy = ((all_y_pred>0.5).byte() == all_y.byte()).float().mean()

In [ ]:
test_loss

In [ ]:
accuracy